In [1]:
from threading import Thread
import pandas as pd
import time

In [2]:
class FindThread(Thread):
    """
    Пример многопоточного  поиска слов из текстового файла
    """
    
    def __init__(self, word, index):
        """Инициализация потока"""
        Thread.__init__(self)
        self.index = index
        self.word = word
    
    def run(self):
        """Запуск потока"""
        
        find = list(df.index[df.Word == self.word])
        if len(find) == 1:
            result.update({(paragraph, self.word) : find[0] })

In [3]:
def main(text):
    """
    Run the program
    """
    for item, word in enumerate(text):
        thread = FindThread(word, item)
        thread.start()

In [4]:
path_dict = 'Index_words_eng.csv'  #  Путь в словарю
filename ='text.txt' # Путь к тексту
df = pd.read_csv(path_dict)
result = {}

In [5]:
with open(filename) as code_book:
        paragraph = 0
        start_time = time.time()
        for line in code_book:
            words = line.strip().lower()
            main(words.split())
            paragraph += 1
        print("--- %s seconds ---" % (time.time() - start_time))
        print(result)

--- 25.570736408233643 seconds ---
{(0, 'atom'): 285, (0, 'block'): 27037, (0, 'basic'): 2303, (0, 'a'): 58, (0, 'be'): 2120, (0, 'up'): 20914, (1, 'a'): 58, (1, 'charge'): 3931, (2, 'nucleus'): 13969, (2, 'atom'): 285, (2, 'up'): 20914, (2, 'number'): 14096, (2, 'it'): 9776, (2, 'a'): 58, (3, 'number'): 14096, (3, 'nuclei'): 13970, (3, 'oxygen'): 14488, (3, 'a'): 58, (3, 'substance'): 17636, (3, 'be'): 2120, (3, 'nuclear'): 14289, (3, 'element'): 5975, (4, 'artist'): 476, (4, 'rendering'): 17077, (4, 'number'): 14096, (4, 'may'): 12537, (4, 'atom'): 285, (4, 'stability'): 18930, (4, 'radioactive'): 17095, (4, 'a'): 58, (4, 'it'): 9776, (6, 'nucleus'): 13969, (6, 'atom'): 285, (6, 'a'): 58, (6, 'be'): 2120, (6, 'neutral'): 14036, (6, 'it'): 9776, (6, 'number'): 14096, (7, 'may'): 12537, (7, 'be'): 2120, (7, 'atom'): 285, (7, 'bind'): 5393, (7, 'up'): 20914, (7, 'a'): 58, (8, 'u.s.'): 20832, (8, 'nuclear'): 14289, (8, 'government'): 7725, (8, 'free'): 6675, (8, 'copyright'): 3129, (8, '